In [1]:
!pip3 install torch torchvision torchaudio

In [2]:
!pip install opencv-python


In [3]:
!pip install flask_cors

In [4]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.1 MB/s eta 0:00:00


In [5]:
import easyocr
from PIL import Image,ImageDraw
import cv2
import matplotlib.pyplot as plt
import re
import os

from flask import Flask, request
from google.cloud import storage
from io import BytesIO
from flask_cors import CORS
from urllib.parse import unquote

In [6]:
 def CariNIK(X):
    for i in X:
        if re.match(r'\d{10,}',i):
            return i

    return ""

def CariNama(X):
    c=0
    for i in X:
        if c==1:
            return i
        if re.match(r'nam[a-z]',i.lower()):
            c=1

    return ""

def CariTTL(X):
  for i in X:
    match = re.search(r'[a-zA-Z, ]+(\d{2}[- ]{1}\d{2}[- ]{1}\d{4})', i)
    if match:
        extracted_format = match.group(1)
        return extracted_format
  return ""

In [7]:
app = Flask(__name__)
CORS(app)

In [8]:
def ktp_detection():
    model = keras.models.load_model('detection_model.h5')
    img_path = 'ktp1.jpeg'
    img = image.load_img(img_path, target_size=(150, 150))  # Adjust target size as needed
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)

    return predictions

In [9]:
reader = easyocr.Reader(['id'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [10]:
#Get OCR Data (still a guess, haven't tried it yet)
def getOCRData(params):
    response = requests.get(params)
    image = Image.open(BytesIO(response.content))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Convert Image to GrayScale
    gaussian = cv2.GaussianBlur(src=gray,ksize=(3, 3),sigmaX=0,sigmaY=0)
    clahe = cv2.createCLAHE(clipLimit=2.00, tileGridSize=(12, 12))
    image = clahe.apply(gaussian)
    _, final_image = cv2.threshold(image, thresh=165, maxval=255, type=cv2.THRESH_TRUNC + cv2.THRESH_OTSU) #Thresholding to Improve Text Quality
    border_image = cv2.copyMakeBorder(
            src=final_image,
            top=20,
            bottom=20,
            left=20,
            right=20,
            borderType=cv2.BORDER_CONSTANT,
            value=(255, 255, 255))
    results = reader.readtext(image)
    List= []
    for result in results:
        List.append(result[1])
    return json.dumps({
      'error': 'false',
      'message': 'Data berhasil diterima!',
      'NIK':CariNIK(List),
      'Nama':CariNama(List),
      'Tgl Lahir':CariTTL(List),
      'Link Photo':params
      })

In [11]:
#Still a guess, haven't tried it yet

@app.route('/OCR',methods=['POST'])
def upload():
    ktp_detection()
    if predictions[0,0] == 0:
        gcpClient = secretmanager.SecretManagerServiceClient()
        keysName = f"projects/xxx/secrets/gcs-key/versions/latest" #perlu diganti
        response = gcpClient.access_secret_version(request={"name": keysName})
        credentials = json.loads(response.payload.data.decode('UTF-8'))
        client = storage.Client.from_service_account_info(credentials)
        file= request.files['images']
        file.save(file.filename)
        bucket_name = "xxx"
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob('fotoktp/{}'.format(file.filename))
        blob.upload_from_filename(file.filename)
        linkFoto = blob.public_url
        os.remove(file.filename)
        test = getOCRData(linkFoto)
        return test

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
